<div style="font-family: cursive, sans-serif;
      background-color: #2ecc71;
      display:full;
      font-size:150%;
      color: white;
      display:fill;
      padding: 10px;
      border-radius: 10px;
      box-shadow: 2px 2px 10px 10px #7fecad">
<h1>The first word</h1>
<p>
      It is very important to select a library to get a good result and try to find the Short Way to Go Smoothly. I will actually try to see if it can be done easily for come with good results. So what, if you have better ideas, Just comments or connect with 
    <a href="https://www.linkedin.com/in/md-hasan-shahriar-08a03b106/">[My LinkedIn link ]</a> any time.
    
</p>
<p>
    ***important library*** <br> 
    1. Numpy <br> 
    2. Pandas <br> 
    3. sklearn <br> 
    4. CV2 <br> 
    5. tensorflow <br> 
</p>
    
</div>

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn import *

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.utils import shuffle
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB3,EfficientNetB2,EfficientNetB1,EfficientNetB7,EfficientNetB0,Xception
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

__<h2>pathlib can easily extract your directory</h2>__

* Make data frame columns:
    + image_link
    + image_Names
    + image_Labels
* Concat and make Dataframe:
    + df1

In [ ]:
img_path=Path('/kaggle/input','alzheimer-parkinson-disease')
image_link=list(img_path.glob(r'**/*.*')) # image link
image_Names=[x.parents[0].stem for x in image_link] #labels name
df=pd.DataFrame(list(zip(image_link,image_Names)),columns=['Link','Name'])

In [ ]:
np.unique(df.Name) # need to drop "Control" bcz of 'CONTROL_AD' & 'CONTROL_PD'

In [ ]:
df=df[df.Name!="CONTROL"] # drop "Control"
df['Labels']=preprocessing.LabelEncoder().fit_transform(df.Name) # create image name labels

In [ ]:
np.unique(df.Labels) #After data set

<h1>Plot</h1>

In [ ]:
df['Name'].value_counts().plot(kind='bar',figsize = (12,8), grid = True,color = 'green')

In [ ]:
fig = plt.figure(1, figsize=(30, 30))
grid = ImageGrid(fig,121, nrows_ncols=(4, 2), axes_pad=0.10)
i = 0
for category_id, category in enumerate(df.Name.unique()):
    for filepath in df[df['Name'] == category]['Link'].values[:2]:
        ax = grid[i]
        img = image.load_img(filepath, (150, 150))
        img=image.img_to_array(img)
        ax.imshow(img/255)
        ax.axis('off')
        if i % 2 == 2 - 1:
            ax.text(200, 111, category, verticalalignment='center',fontsize=20,color = 'k')
        
            
        i+=1
    
plt.show();

* <h2>Split labels,class and tarin,test:</h2>
+    <h3>train Data </h3>
+   <h3>test data</h3>

In [ ]:
x=df.Link
y=df.Labels
x_data,y_data=shuffle(x,y,random_state=42) #data shuffle for better result
xtrain,xtest,ytrain,ytest=model_selection.train_test_split(x_data, y_data, test_size=0.1, random_state=42)

<h2>Train and Test image to data format:</h2>

In [ ]:
imsize=150
X_train=[]
X_test=[]
for i in xtrain:
    # for train
    img_pre=cv2.imread(str(i))
    img_pre=cv2.resize(img_pre, (imsize,imsize))
    X_train.append(img_pre)
    
    
for j in xtest:
    # for test
    img_pre1=cv2.imread(str(j))
    img_pre1=cv2.resize(img_pre1, (imsize,imsize))
    X_test.append(img_pre1)
    
    
X_train=np.array(X_train)
X_test=np.array(X_test)
ytrain=to_categorical(ytrain)
ytest=to_categorical(ytest)

__<h2>Create model with EfficientNetB7 </h2>__

In [ ]:
efficient = EfficientNetB7(weights='imagenet',include_top = False, input_shape=(imsize, imsize, 3))

model = efficient.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(0.5)(model)
model = tf.keras.layers.Dense(4, activation = "softmax")(model)
model = tf.keras.models.Model(inputs = efficient.input , outputs = model)

model.compile(loss = 'categorical_crossentropy', 
             optimizer = "Adam", 
              metrics=["accuracy"]
             )

tensorboard = TensorBoard(log_dir="logs")
checkpoint = ModelCheckpoint("efficientB7.h5", 
                            monitor='val_accuracy', 
                            mode = "auto", 
                            verbose = 1,
                            save_best_only =True)
reduce_LR = ReduceLROnPlateau(monitor="val_accuracy", 
                             factor = 0.3, 
                             patience = 2, 
                             min_delta=0.001, 
                             mode = "auto", 
                            verbose = 1)

__<h2>Fit model with EfficientNetB7 </h2>__

In [ ]:
history = model.fit(X_train, ytrain,
                  validation_data =(X_test,ytest), 
                  epochs = 15, 
                  verbose =1, 
                  batch_size = 32, 
                  callbacks = [tensorboard, checkpoint, reduce_LR])

__<h2>For data frame loss and accuracy</h2>__

* Make data frame for df_accuracy:
    + loss
    + accuracy
    + val_loss
    + val_accuracy


In [ ]:
df_accuracy=pd.DataFrame()
df_accuracy['Loss']=history.history['loss']
df_accuracy['Value Loss']=history.history['val_loss']
df_accuracy['Accuracy']=history.history['accuracy']
df_accuracy['Value Accuracy']=history.history['val_accuracy']

df_accuracy

__<h2>Plot for best performance with EfficientNetB7 </h2>__

In [ ]:
df_accuracy.sort_values('Loss',ascending=False).head(1).plot(kind='bar',figsize = (12,8))

__<h2>Visualization for accuracy</h2>__

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("Model Accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

plot_hist(history)

__<h2>Visualization for loss</h2>__

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["loss"])
    plt.plot(hist.history["val_loss"])
    plt.title("Model Loss")
    plt.ylabel("Loss")
    plt.xlabel("epoch")
    plt.legend(["loss", "value Loss"], loc="upper left")
    plt.show()


plot_hist(history)

__<h2>Conclusion:</h2>__
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">

<p style="padding: 10px;color:white;">
Got 99.17 accuracy. it's not easy task. 1st you need to remove contradiction class 'CONTROL' becouse of separet 'CONTROL_PD'/'CONTROL_AD'. Otherwise model loss 20% to 30% accuracy. After that, if you hav e any suggetion to please comments.
</p>
</div>